In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from library import utils, config, cascades

In [2]:
with open('./results/nopricechange/7-9/traderIDs_501_7200_100_new2threshold_fixedhurst.pkl', 'rb') as f:
    traderIDs = pickle.load(f)

Importing MATLAB


In [3]:
stockPool = np.load("./results/nopricechange/7-9/stockPool_501_7200_100_new2threshold_fixedhurst.npy")
hurstPool = np.load("./results/nopricechange/7-9/hurstPool_501_7200_100_new2threshold_fixedhurst.npy")
Ttransactions = pd.read_csv("./results/nopricechange/7-9/transactions_501_7200_100_new2threshold_fixedhurst.csv")
TtotalOrders = pd.read_csv("./results/nopricechange/7-9/totalOrders_501_7200_100_new2threshold_fixedhurst.csv")

# Cascades with time limit

In [5]:
def findPortfOrderCascades(TtotalOrders, portf, t0, maxSep = 1):
    """
    finds all subsequent activity traded by a portfolio (time threshold)
    input TtotalOrders, the portfolio, and minimum time
    FOR REAL CASCADES Aug18
    """

    portfOrders = TtotalOrders[TtotalOrders['portfolio']==portf]
    portfOrders = portfOrders[portfOrders['time']>=t0]
    if len(portfOrders)>0:
        ToS = portfOrders['time'].unique()
        sep = np.diff(ToS)
        #print(sep)
        #print(max(sep))
        keySep = np.where(sep>maxSep)[0]
        #print(keySep)
        if len(keySep)>0:
            endtime = ToS[keySep[0]] #end of this cascade
        else:
            endtime = ToS[-1]
        portfCascade = portfOrders[portfOrders['time']<=endtime]
        return portfCascade
    else:
        return pd.DataFrame()

In [6]:
cascades = {}
numCascade = 0
cascadeStocks = {}
cascadePortfs = {}
cascadeTime = {}
cascadeTf = {}
cascades = {}
maxtimeeff = 2
while len(TtotalOrders) > 0:
#     totalTimeOrders = TtotalOrders[TtotalOrders['time']==t]
    # seed with first portfolio
    print("OG Orders left: ", len(TtotalOrders))
    seed = (TtotalOrders.iloc[0]['portfolio'])
    t0 = TtotalOrders.iloc[0]['time']
    seedCascade = findPortfOrderCascades(TtotalOrders, seed, t0, maxSep = 1)
    cascadeStocks[numCascade] = np.asarray(seedCascade['stock'])
    cascadePortfs[numCascade] = np.asarray(seed)
    cascadeTime[numCascade] = t0
    cascades[numCascade] = seedCascade
    stockTimes = {}
    for row in seedCascade.iterrows():
        stockTimes[row[1]['stock']] = row[1]['time'] + abs(row[1]['order'])*maxtimeeff
    assert len(seedCascade) > 0
    TtotalOrders = TtotalOrders[~TtotalOrders.isin(seedCascade)].dropna()
    # NEW VERSION (PORTFOLIO PERSPECTIVE)
    for childPortf in list(TtotalOrders['portfolio'].unique()):
        match = False
        if any(np.isin(cascadeStocks[numCascade],traderIDs[childPortf].stocks)):
#             print("match: ", childPortf)
            if childPortf not in cascadePortfs[numCascade]:
                childPortfCascade = findPortfOrderCascades(TtotalOrders,childPortf,t0, maxSep = 1)
                for row in childPortfCascade.iterrows():
                    if row[1]['stock'] in stockTimes:
                        if row[1]['time'] <= stockTimes[row[1]['stock']]:
                            match = True
                            break 
                if match == True:        
                    cascadePortfs[numCascade] = np.append(cascadePortfs[numCascade], childPortf)
                    cascades[numCascade] = pd.concat([cascades[numCascade],childPortfCascade])
                    TtotalOrders = TtotalOrders[~TtotalOrders.isin(childPortfCascade)].dropna()
#                     for childStock in list(childPortfCascade['stock'].unique()):
#                         if childStock not in cascadeStocks[numCascade]:
#                             cascadeStocks[numCascade] = np.append(cascadeStocks[numCascade], childStock)
                    for row in childPortfCascade.iterrows():
                        if row[1]['stock'] in stockTimes:
                            stockTimes[row[1]['stock']] += abs(row[1]['order'])*maxtimeeff
                        else:
                            stockTimes[row[1]['stock']] = row[1]['time'] + abs(row[1]['order'])*maxtimeeff
                            cascadeStocks[numCascade] = np.append(cascadeStocks[numCascade], row[1]['stock'])
    print("Cascade length: ",len(cascades[numCascade]))
    print("New Orders left: ", len(TtotalOrders))
    cascadeTf[numCascade] = cascades[numCascade]['time'].max()
    print("making new cascade")
    numCascade += 1
    


    
print(cascadeStocks)
print(cascadePortfs)
print(cascadeTime)

OG Orders left:  7088505
Cascade length:  7101
New Orders left:  7081404
making new cascade
OG Orders left:  7081404
Cascade length:  7416
New Orders left:  7073988
making new cascade
OG Orders left:  7073988
Cascade length:  6969
New Orders left:  7067019
making new cascade
OG Orders left:  7067019
Cascade length:  6771
New Orders left:  7060248
making new cascade
OG Orders left:  7060248
Cascade length:  2
New Orders left:  7060246
making new cascade
OG Orders left:  7060246
Cascade length:  6664
New Orders left:  7053582
making new cascade
OG Orders left:  7053582
Cascade length:  6545
New Orders left:  7047037
making new cascade
OG Orders left:  7047037
Cascade length:  1
New Orders left:  7047036
making new cascade
OG Orders left:  7047036
Cascade length:  6819
New Orders left:  7040217
making new cascade
OG Orders left:  7040217
Cascade length:  20
New Orders left:  7040197
making new cascade
OG Orders left:  7040197
Cascade length:  6582
New Orders left:  7033615
making new casc

Cascade length:  4
New Orders left:  6742285
making new cascade
OG Orders left:  6742285
Cascade length:  4273
New Orders left:  6738012
making new cascade
OG Orders left:  6738012
Cascade length:  4
New Orders left:  6738008
making new cascade
OG Orders left:  6738008
Cascade length:  7
New Orders left:  6738001
making new cascade
OG Orders left:  6738001
Cascade length:  35
New Orders left:  6737966
making new cascade
OG Orders left:  6737966
Cascade length:  12
New Orders left:  6737954
making new cascade
OG Orders left:  6737954
Cascade length:  34
New Orders left:  6737920
making new cascade
OG Orders left:  6737920
Cascade length:  4833
New Orders left:  6733087
making new cascade
OG Orders left:  6733087
Cascade length:  1
New Orders left:  6733086
making new cascade
OG Orders left:  6733086
Cascade length:  4430
New Orders left:  6728656
making new cascade
OG Orders left:  6728656
Cascade length:  9
New Orders left:  6728647
making new cascade
OG Orders left:  6728647
Cascade l

KeyboardInterrupt: 